# Open Reading Frames
https://rosalind.info/problems/orf/b

### Practice 

In [1]:
from util import read_file_into_lines
from util import text_to_fasta
from util import codon_dict

In [2]:
s = "AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"

In [3]:
counter = 0
peptide = ""
start_codon = "ATG"
stop_codons = ["TAG", "TGA", "TAA"]

In [4]:
# this did not work but it was a preliminary thinking
for i in range(len(s)):
    if s[i:i+3] == start_codon:
        print("start codon", s[i:i+3])       
    elif s[i:i+3] in stop_codons:
        print("end codon", s[i:i+3])
        peptide += s[i:i+3]
        break
    else: 
        print(s[i:i+3])
        peptide += s[i:i+3]

AGC
GCC
CCA
CAT
start codon ATG
TGT
GTA
end codon TAG


In [5]:
def find_candidate_string(dna_string):
    current_list = []
    for i in range(len(dna_string)):
        if dna_string[i:i+3] == "ATG":
            mrna = ""
            for j in range(i, len(dna_string), 3):
                codon = dna_string[j:j+3]
                if codon in stop_codons:
                    current_list.append(mrna)
                    break
                else: 
                    mrna += dna_string[j:j+3]
    return(current_list)

In [6]:
    mrna_list = find_candidate_string(s)

In [7]:
mrna_list

['ATG',
 'ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA',
 'ATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA']

In [8]:
sc = s[::-1].replace("A","t").replace("T", "a").replace("C", "g").replace("G", "c").upper()

mrna_list.extend(find_candidate_string(sc))
print(mrna_list)

['ATG', 'ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA', 'ATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA', 'ATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTCTAATCCAAGTCGCGGGGTCATCCCCATGTAACCTGAGT', 'ATG']


In [9]:
def translate_mrna_rna(mrna_list):
    rna_list = []
    for i in mrna_list:
        rna = ""
        for nucl in i:
            if nucl == "T":
                rna = rna + "U"
            else:
                rna = rna + nucl
        rna_list.append(rna)
    return(rna_list)

In [10]:
rna_list = translate_mrna_rna(mrna_list)

In [11]:
rna_list

['AUG',
 'AUGGGGAUGACCCCGCGACUUGGAUUAGAGUCUCUUUUGGAA',
 'AUGACCCCGCGACUUGGAUUAGAGUCUCUUUUGGAA',
 'AUGCUACUCGGAUCAUUCAGGCUUAUUCCAAAAGAGACUCUAAUCCAAGUCGCGGGGUCAUCCCCAUGUAACCUGAGU',
 'AUG']

In [12]:
def from_rna_to_protein(rna_list):
    protein_list = []
    for i in rna_list:
        counter = 0 
        aa = ""
        for codon in i:
            triplet = i[counter:counter + 3] 
            if triplet == '': 
                break
            else:
                aa = aa + codon_dict[triplet] 
                counter = counter + 3
        protein_list.append(aa)
    return(protein_list)

In [13]:
from_rna_to_protein(rna_list)

['M', 'MGMTPRLGLESLLE', 'MTPRLGLESLLE', 'MLLGSFRLIPKETLIQVAGSSPCNLS', 'M']

### All together 


In [29]:
fasta_file = read_file_into_lines("rosalind_orf.txt")

In [30]:
#now we can put all the functions together into one program so that we can input a rasta file and have a resulting list of protein chains

def fasta_to_protein(fasta_file):
    dna_list = text_to_fasta(fasta_file)
    for dna_string in dna_list.values(): 
        mrna_list = find_candidate_string(dna_string)
        sc = dna_string[::-1].replace("A","t").replace("T", "a").replace("C", "g").replace("G", "c").upper()
        mrna_list.extend(find_candidate_string(sc))
    rna_list = translate_mrna_rna(mrna_list)
    protein_list = from_rna_to_protein(rna_list)
    return protein_list

In [31]:
final_solution = fasta_to_protein(fasta_file)
for protein in final_solution:
    print(protein)

MPSLKSYGEQK
MRSH
MLAAERARRSSDTSTANCVLPSC
MSLLHAVNRNLTSPAPLILDPDHELSAGPLHIGKITAEVSVQPRVG
M
MIILGGGGIQKGQNFVYRPIYHHCD
MHYPPLLAQEKFPTSSGSRLAFP
MSLLITITFCVTPLGSEPMVVVRPVTHVVAR
MVVVRPVTHVVAR
MLGGSH
MHPLPIAKVQDGLGFVLV
MSPSMNRTTMRLITMVIDRTINKVLTFLNSSAT
MNRTTMRLITMVIDRTINKVLTFLNSSAT
MRLITMVIDRTINKVLTFLNSSAT
MVIDRTINKVLTFLNSSAT
MSYVIANVRDATMARCS
MRQWRDVAKQRLFERLAKRSKSRCLATFFLANSRLNAYLGCNLTNVKRSSRQFVVGV
MARCS
M
M
MCPATRLPV
ML
MGTHPR


### How do I avoid having double entries for proteins that already exist? 